<a href="https://colab.research.google.com/github/hazrakeruboO/DS-Colabs/blob/main/Copy_of_SQL_Mini_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**SQL Mini-Project**

To aid a US presidential candidate, we would like to understand the states where he/she should focus campaign efforts in order to secure a win the upcoming presidential elections.

In each of the 51 states, there is a given number of grand electors to win and the presidential candidate receiving the most local votes wins ALL the Grand Electors in that state.

However,it is impotant to note that the number of grand electors is not exactly proportional to the population.

Therefore, the objective of this study is identify which states we need to prioritize in our campaigns to ensure the we win the most votes and/or which states have the highest grand electors per capita.

---



**Connecting the Database**

In [ ]:
# Loading sql extension
%load_ext sql

# Connecting to sqlite in our memory

%sql sqlite://

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


'Connected: @None'

**Loading required Python Library**

In [ ]:
import pandas as pd
import numpy as np

**Loading Dataset**

In [ ]:
# Loading and Previewing Grand Electors dataset

with open('GrandElectors_by_state.csv', 'r') as elec:
   GrandElectors = pd.read_csv(elec, index_col=0, encoding='utf-8')

%sql DROP TABLE if EXISTS GrandElectors;
%sql PERSIST GrandElectors;
%sql SELECT * FROM GrandElectors LIMIT 5;

 * sqlite://
Done.
 * sqlite://
 * sqlite://
Done.


State,GrandElectors
Alabama,9
Alaska,3
Arizona,11
Arkansas,6
California,55


In [ ]:
# Loading Population Dataset
with open('Population_by_state.csv', 'r') as pop:
  Population = pd.read_csv(pop, index_col=0, encoding='utf-8')


%sql DROP TABLE IF EXISTS Population
%sql PERSIST Population
%sql SELECT * FROM Population LIMIT 5;



 * sqlite://
Done.
 * sqlite://
 * sqlite://
Done.


State,Population
CALIFORNIA,39144818
TEXAS,27469114
FLORIDA,20271272
NEW YORK,19795791
ILLINOIS,12859995


The first thing we shall do is to join the two datasets using SQL on the common columns state. 

However, note that the name of the state in the population dataset is in all caps. This might cause an issue in the JOIN clause. We need to make sure that both columns are the same before the JOIN clause.

Additionally, we need to the joined table an independent table in order to conduct analysis on it later.

In [ ]:
# Making States values in GrandElectors dataset uppercase to match the population dataset

%%sql
UPDATE GrandElectors SET State=UPPER(State)

 * sqlite://
51 rows affected.


[]

In [ ]:
# Creating new table and joining Datasets 

%%sql
CREATE TABLE US_Elections AS SELECT Population.State, GrandElectors.GrandElectors, Population
FROM Population INNER JOIN GrandElectors
ON Population.State=GrandElectors.State;

 * sqlite://
Done.


[]

In [ ]:
# Preview new table

%%sql
SELECT * FROM US_Elections LIMIT 5;

 * sqlite://
Done.


State,GrandElectors,Population
CALIFORNIA,55,39144818
TEXAS,38,27469114
FLORIDA,29,20271272
NEW YORK,29,19795791
ILLINOIS,20,12859995


We been asked to change the District of Colombia in our new table to 'DC'

In [ ]:
# Updating 'District of Columbia' as 'DC'

%%sql
UPDATE US_Elections
SET State = 'DC'
WHERE State = 'DISTRICT OF COLUMBIA';

 * sqlite://
1 rows affected.


[]

Next, we need to create a new column and update its values to reflect the ratio. The resulting values will define our priority states with the state with the highest ration being the most important state.

In [ ]:
# Creating new column

%%sql
ALTER TABLE US_Elections
ADD Ratio;

 * sqlite://
Done.


[]

In [ ]:
# Updating the values in the new column

%%sql
UPDATE US_Elections
SET "Ratio" = (US_Elections.GrandElectors * 1.0 / US_Elections.Population * 1.0);

 * sqlite://
51 rows affected.


[]

From the table above we have seen that Wyoming, Vermont, DC and Alaska are top of our priority states. We now need to compute the running total from the state with the highest ratio.

In [ ]:
# Calculating the running total and creating a new table with the new columns.

%%sql

CREATE TABLE US_Elections_Updated AS 
  SELECT t.State,
       t.GrandElectors,
       t.Ratio,
       (SELECT SUM(x.GrandElectors)
       FROM US_Elections x
       WHERE t.Ratio <= x.Ratio) AS Running_Total
FROM US_Elections t
ORDER BY t.Ratio DESC;

SELECT * FROM US_Elections_Updated;

 * sqlite://
Done.
Done.


State,GrandElectors,Ratio,Running_Total
WYOMING,3,5.118519314732634e-06,3
VERMONT,3,4.792010759661493e-06,6
DC,3,4.462771559649405e-06,9
ALASKA,3,4.0626625065002604e-06,12
NORTH DAKOTA,3,3.963394092164766e-06,15
RHODE ISLAND,4,3.7868101615263876e-06,19
SOUTH DAKOTA,3,3.4945932817609025e-06,22
DELAWARE,3,3.1714686225466047e-06,25
MAINE,4,3.009039153617467e-06,29
NEW HAMPSHIRE,4,3.0061445594795764e-06,33


Next, we need to calculate our threshold which is equal to half the total GrandElectors.


In [ ]:
# Calculating the Threshold
%%sql
SELECT SUM("GrandElectors") * 0.5 AS Threshold FROM US_Elections_Updated;

 * sqlite://
Done.


Threshold
269.0


Finally, we need to fiter the data to show use states where the running total is under the threshold and add one state above the threshold.

In [ ]:
%%sql
CREATE TABLE Priority_States AS
SELECT State, GrandElectors, Ratio, Running_Total FROM US_Elections_Updated
  WHERE Running_Total <= 269.0 OR Running_Total = 271;

SELECT * FROM Priority_States;

 * sqlite://
(sqlite3.OperationalError) no such table: US_Elections_Updated
[SQL: CREATE TABLE Priority_States AS
SELECT State, GrandElectors, Ratio, Running_Total FROM US_Elections_Updated
  WHERE Running_Total <= 269.0 OR Running_Total = 271;]
(Background on this error at: http://sqlalche.me/e/14/e3q8)


In [ ]:
# Ordering the top 10 priority states

%%sql
SELECT * FROM US_Elections 
ORDER BY Ratio DESC;

 * sqlite://
Done.


State,GrandElectors,Population,Ratio
WYOMING,3,586107,5.118519314732634e-06
VERMONT,3,626042,4.792010759661493e-06
DC,3,672228,4.462771559649405e-06
ALASKA,3,738432,4.0626625065002604e-06
NORTH DAKOTA,3,756927,3.963394092164766e-06
RHODE ISLAND,4,1056298,3.7868101615263876e-06
SOUTH DAKOTA,3,858469,3.4945932817609025e-06
DELAWARE,3,945934,3.1714686225466047e-06
MAINE,4,1329328,3.009039153617467e-06
NEW HAMPSHIRE,4,1330608,3.0061445594795764e-06


In [ ]:
%%sql
SELECT COUNT(*) FROM Priority_States

 * sqlite://
Done.


COUNT(*)
40


**Conclusions**



1.   The campaign team should focus their efforts on the 40 most influential states in terms of grand electors per capita. 
2.   Winning majority local votes in these states will sway the elections in the candidates favour, giving him/her a good chance at winning the elections


